In [6]:
import numpy as np
import cv2
from skimage import color

def read_video_cv2(n_frames=10):
    cap = cv2.VideoCapture("./survelliance/09152008flight2tape1_7.mpg")
    all = []
    i = 0
    while cap.isOpened() and i < n_frames:
        ret, frame = cap.read()
        img = color.rgb2gray(frame)
        arr = np.array(img)
        all.append(arr)
        i += 1
    return np.array(all)

video_seg = read_video_cv2()
print(video_seg.shape)

(10, 480, 720)


In [3]:
import tensorflow as tf

K = np.ones((30 * 20, 10))
A = np.ones((30 * 20, 10))
C = np.ones((30 * 20, 10))
U = np.ones((30 * 20, 10))
Lambda1 = np.ones((30 * 20, 10))
Lambda2 = np.ones((30 * 20, 10))

def k_subproblem(K, A, C, Lambda1):
    mu = 0.1
    K_next = tf.norm(K, ord='euclidean') + mu * (tf.norm(K - (A - C + Lambda1/mu), ord='euclidean') ** 2)
    return K_next

NameError: name 'np' is not defined

In [ ]:
K_next = k_subproblem(K, A, C, Lambda1)

In [ ]:
# the second parameter of shrinkage operator should be defined as param_lambda/(2*mu)

def shrinkage_operator(x):
    # the shape of x is 1080 * 1920
    epsilon = np.zeros((600, 10))
    para1 = np.sign(x)
    para2 = np.maximum((np.fabs(x)-epsilon), np.zeros((600, 10)))
    return np.multiply(para1, para2)

In [ ]:
shrinkage = shrinkage_operator(K)

In [ ]:
shrinkage.shape

In [ ]:
print(video_seg[0].shape[1])
frame = video_seg.shape[2]
np.max((video_seg[0].shape[0], video_seg[0].shape[1]))
param_lambda = 8 / np.sqrt( np.max((video_seg[0].shape[0], video_seg[0].shape[1])) * frame)
print(param_lambda)

def c_subproblem(C, A, K, U, Lambda1, Lambda2):
    mu = 0.01
    C_next_1 = Lambda1 * tf.norm(C, ord=1) + mu * (tf.norm(C - ((A - K + U)/2 + (Lambda1 + Lambda2)/2*mu), ord='euclidean') ** 2)
    C_next_2 = shrinkage_operator((A - K - U)/2 + (Lambda1 + Lambda2)/2*mu)
    return C_next_1, C_next_2

In [ ]:
c_subproblem(C, A, K, U, Lambda1, Lambda2)

In [ ]:
def tv_norm(input_matrix):
    res_matrix = np.zeros((input_matrix.shape[0], input_matrix.shape[1]))
    print(res_matrix.shape)
    for i in range(0, input_matrix.shape[0]):
        for j in range(0, input_matrix.shape[1]):
            if i+1 < 480 and j+1 < 720:
                res_matrix[i][j] = np.sqrt(np.abs(input_matrix[i+1][j] - input_matrix[i][j]) ** 2 + np.abs(input_matrix[i][j+1] - input_matrix[i][j]) ** 2)
    return res_matrix
        
    
def update_multiplier(Lambda1, Lambda2):
    Lambda1_next = Lambda1 + mu * (A - K - C)
    Lambda2_next = Lambda2 + mu * (U - C)
    return Lambda1_next, Lambda2_next
    

In [ ]:
import torch
print(U.shape)


def tv_norm_seg(input_matrix):
    res_matrix = np.zeros((input_matrix.shape[0], input_matrix.shape[1]))
    print(torch.tensor(res_matrix).shape)
    for i in range(0, input_matrix.shape[0]):
        for j in range(0, input_matrix.shape[1]):
            if i+1 < 600 and j+1 < 10:
                res_matrix[i][j] = np.sqrt(np.abs(input_matrix[i+1][j] - input_matrix[i][j]) ** 2 + np.abs(input_matrix[i][j+1] - input_matrix[i][j]) ** 2)
    return res_matrix

mu = 10 ** (-4)
def u_subproblem(U, Lambda2, C):
    U_next_1 = tv_norm_seg(U) + mu/2 * tf.norm(U - (C - Lambda2/mu), ord='euclidean') ** 2
    U_next_2 = shrinkage_operator(tf.norm(U - (C - Lambda2/mu), ord='euclidean') ** 2)
    return U_next_1, U_next_2

u_subproblem(U, Lambda2, C)

In [ ]:
update_multiplier(Lambda1, Lambda2)

In [ ]:
print(video_seg[0].shape)
tv_norm(video_seg[0])

In [ ]:
print(C.shape)
u_subproblem(U, Lambda2, C)

In [ ]:
update_multiplier

In [ ]:
# the input is K, C, U, lambda1, Lambda2 with shape mh * p, mu, rho, mu_max, k, the shape of A is mh * p 
def LRl1TV(K, C, A, U, Lambda1, Lambda2):
    mu = 10 ** (-4)
    mu_max = 10 ** 4
    k = 0
    terminate_flag = False
    iter_num = 0
    rho = 5
    
    while iter_num < 150:
        iter_num += 1
        print("this is " + str(iter_num) + "th iteration")
        K_next = k_subproblem(K, A, C, Lambda1)
        C_next = c_subproblem(C, A, K_next, U, Lambda1, Lambda2)
        print(type(U[0]))
        if type(U)== tuple:
            U = u_subproblem(U[0], Lambda2, C_next[0])
        else:
            U = u_subproblem(U, Lambda2, C_next[0])
        mu = np.min((rho * mu, mu_max))
        Lambda1, Lambda2 = update_multiplier(Lambda1, Lambda2)
        
        terminate_flag = False
       
        terminate_flag = tf.norm(A - K - C, ord=2) ** 2 <= 10 ** (-8) * (tf.norm(A, ord='euclidean') ** 2)
        if terminate_flag == True:
            break
    return K, C
        

In [ ]:
K_res, C_res = LRl1TV(K, C, A, U, Lambda1, Lambda2)

In [ ]:
from PIL import Image

print(B_res.shape)
im_output_sparse = Image.fromarray(F_res[:, :, 0].astype(np.uint8))
if im_output_sparse.mode != 'RGB':
    im_output_rank = im_output_sparse.convert('RGB')
im_output_rank.save("./res/tvrpca.png")